# Tollama Agent Demo

This notebook shows an agent-oriented flow using Tollama LangChain tools.

Prerequisites:
- `pip install -e ".[langchain]"`
- Daemon running: `tollama serve`


In [ ]:
from tollama.skill import get_tollama_tools

tools = {tool.name: tool for tool in get_tollama_tools()}
sorted(tools)


In [ ]:
# 1) Discover candidate models
recommend_payload = tools["tollama_recommend"].invoke({
    "horizon": 24,
    "freq": "D",
    "has_past_covariates": True,
    "has_future_covariates": True,
    "covariates_type": "numeric",
    "top_k": 3,
})
recommend_payload


In [ ]:
# 2) Compare top two candidates on the same series
recommended = recommend_payload.get("recommendations", [])[:2]
candidate_models = [item["model"] for item in recommended]
if not candidate_models:
    candidate_models = ["mock", "chronos2"]

compare_payload = tools["tollama_compare"].invoke({
    "request": {
        "models": candidate_models,
        "horizon": 2,
        "series": [{
            "id": "s1",
            "freq": "D",
            "timestamps": [
                "2025-01-01",
                "2025-01-02",
                "2025-01-03",
                "2025-01-04",
            ],
            "target": [10.0, 11.0, 12.0, 13.0],
            "actuals": [14.0, 15.0],
        }],
        "parameters": {
            "metrics": {
                "names": ["mae", "rmse"],
            },
        },
        "options": {},
    },
})
compare_payload


The `tollama_recommend` and `tollama_compare` tools are suitable for agent planning loops:
- recommend candidate models from metadata/capabilities
- compare candidate outputs/metrics under the same request
- select the best model for downstream forecasting
